## Exercise 1

### Import libraries:

In [55]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

### Scrape the Wikipedia page in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe:

#### scraping table using BeautifulSoup:

In [31]:
url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&diff=942655364&oldid=935851093"
source = requests.get(url).text
soup = bs(source, 'lxml')

#### analysing imported data using BeautifulSoup:

In [ ]:
#print(soup.prettify())

#### create empty dataframe: 

In [43]:
col = ['Postalcode','Borough','Neighborhood']
df_canada = pd.DataFrame(columns = col)
df_canada

,Postalcode,Borough,Neighborhood


#### finding the table using BeautufulSoup:

In [44]:
find_table = soup.find('div', class_='mw-parser-output')
find = find_table.table.tbody

#### analysing table using BeautifulSoup:

In [ ]:
#print(find_table.prettify())
#print(find.prettify())

#### finding Postalcode, Borough, Neighborhood:

In [49]:
postal_code = 0
borough = 0
neighborhood = 0

for tr in find.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postal_code = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')

#### filling the empty dataframe:

In [51]:
df_canada = df_canada.append({'Postalcode': postal_code,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)
#df_canada

#### Cleaning the dataframe:

######  - Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

###### - If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

######  - More than one neighborhood can exist in one postal code area. 

In [ ]:
df_canada = df_canada[df_canada.Borough!='Not assigned']

df_canada = df_canada[df_canada.Borough!= 0]

df_canada.reset_index(drop = True, inplace = True)

i = 0
for i in range(0,df_canada.shape[0]):
    if df_canada.iloc[i][2] == 'Not assigned':
        df_canada.iloc[i][2] = df_canada.iloc[i][1]
        i = i+1
                 
            
df_toronto = df_canada.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

In [53]:
df_toronto.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


##### In the last cell of the notebook, use the .shape method to print the number of rows of your dataframe:

In [54]:
print(df_toronto.shape)

(103, 3)
